## Library imports

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import librosa
import soundfile as sf
import pickle
import os
import re

In [2]:
from preprocessing_pipeline import *
from util import *
from util import DATA_ADDRESS
DATA_ADDRESS 

'./data/'

In [3]:
from tqdm import tqdm
tqdm.pandas()

In [4]:
FILE_DIR = DATA_ADDRESS+"VOiCES_devkit/"
target_folder = "5seconds-16k-source"
TARGET_DIR = os.path.join(DATA_ADDRESS, target_folder)

## Read original file

In [5]:
df_raw = pd.read_pickle(data_address+"df_merged.pkl")
df_raw

,index,chapter,degrees,distractor,filename,gender,mic,query_name,room,segment,source,speaker,transcript,noisy_length,noisy_sr,noisy_time,source_length,source_sr,source_time,category
0,0,32639,160,babb,distant-16k/speech/test/rm4/babb/sp4899/Lab41-...,F,1,Lab41-SRI-VOiCES-rm4-babb-sp4899-ch032639-sg00...,rm4,29,source-16k/test/sp4899/Lab41-SRI-VOiCES-src-sp...,4899,and i got on with the lesson very badly i am s...,255040,16000,15.940,255040,16000,15.940,test
1,1,32639,160,babb,distant-16k/speech/test/rm4/babb/sp4899/Lab41-...,F,5,Lab41-SRI-VOiCES-rm4-babb-sp4899-ch032639-sg00...,rm4,29,source-16k/test/sp4899/Lab41-SRI-VOiCES-src-sp...,4899,and i got on with the lesson very badly i am s...,255040,16000,15.940,255040,16000,15.940,test
2,2,32658,70,babb,distant-16k/speech/test/rm4/babb/sp4899/Lab41-...,F,5,Lab41-SRI-VOiCES-rm4-babb-sp4899-ch032658-sg00...,rm4,12,source-16k/test/sp4899/Lab41-SRI-VOiCES-src-sp...,4899,and ran upstairs to put away the host of littl...,258000,16000,16.125,258000,16000,16.125,test
3,3,32658,70,babb,distant-16k/speech/test/rm4/babb/sp4899/Lab41-...,F,1,Lab41-SRI-VOiCES-rm4-babb-sp4899-ch032658-sg00...,rm4,12,source-16k/test/sp4899/Lab41-SRI-VOiCES-src-sp...,4899,and ran upstairs to put away the host of littl...,258000,16000,16.125,258000,16000,16.125,test
4,4,130550,10,babb,distant-16k/speech/test/rm4/babb/sp1447/Lab41-...,F,1,Lab41-SRI-VOiCES-rm4-babb-sp1447-ch130550-sg00...,rm4,26,source-16k/test/sp1447/Lab41-SRI-VOiCES-src-sp...,1447,he was inexhaustibly voluble,58720,16000,3.670,58720,16000,3.670,test
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19195,12795,162129,160,none,distant-16k/speech/train/rm4/none/sp2673/Lab41...,M,1,Lab41-SRI-VOiCES-rm4-none-sp2673-ch162129-sg00...,rm4,27,source-16k/train/sp2673/Lab41-SRI-VOiCES-src-s...,2673,and away with it we ought all to say if social...,239840,16000,14.990,239840,16000,14.990,train
19196,12796,75242,170,none,distant-16k/speech/train/rm4/none/sp1212/Lab41...,F,5,Lab41-SRI-VOiCES-rm4-none-sp1212-ch075242-sg00...,rm4,16,source-16k/train/sp1212/Lab41-SRI-VOiCES-src-s...,1212,and zip dropped from the boy's arms not into t...,269040,16000,16.815,269040,16000,16.815,train
19197,12797,75242,170,none,distant-16k/speech/train/rm4/none/sp1212/Lab41...,F,1,Lab41-SRI-VOiCES-rm4-none-sp1212-ch075242-sg00...,rm4,16,source-16k/train/sp1212/Lab41-SRI-VOiCES-src-s...,1212,and zip dropped from the boy's arms not into t...,269040,16000,16.815,269040,16000,16.815,train
19198,12798,14653,160,none,distant-16k/speech/train/rm4/none/sp1212/Lab41...,F,1,Lab41-SRI-VOiCES-rm4-none-sp1212-ch014653-sg00...,rm4,3,source-16k/train/sp1212/Lab41-SRI-VOiCES-src-s...,1212,for for the sake of my reputation i suggested ...,256080,16000,16.005,256080,16000,16.005,train


In [6]:
print(df_raw.iloc[0])

index                                                            0
chapter                                                      32639
degrees                                                        160
distractor                                                    babb
filename         distant-16k/speech/test/rm4/babb/sp4899/Lab41-...
gender                                                           F
mic                                                              1
query_name       Lab41-SRI-VOiCES-rm4-babb-sp4899-ch032639-sg00...
room                                                           rm4
segment                                                         29
source           source-16k/test/sp4899/Lab41-SRI-VOiCES-src-sp...
speaker                                                       4899
transcript       and i got on with the lesson very badly i am s...
noisy_length                                                255040
noisy_sr                                                     1

In [7]:
df_raw['source']

0        source-16k/test/sp4899/Lab41-SRI-VOiCES-src-sp...
1        source-16k/test/sp4899/Lab41-SRI-VOiCES-src-sp...
2        source-16k/test/sp4899/Lab41-SRI-VOiCES-src-sp...
3        source-16k/test/sp4899/Lab41-SRI-VOiCES-src-sp...
4        source-16k/test/sp1447/Lab41-SRI-VOiCES-src-sp...
                               ...                        
19195    source-16k/train/sp2673/Lab41-SRI-VOiCES-src-s...
19196    source-16k/train/sp1212/Lab41-SRI-VOiCES-src-s...
19197    source-16k/train/sp1212/Lab41-SRI-VOiCES-src-s...
19198    source-16k/train/sp1212/Lab41-SRI-VOiCES-src-s...
19199    source-16k/train/sp1212/Lab41-SRI-VOiCES-src-s...
Name: source, Length: 19200, dtype: object

In [8]:
len(df_raw['source'].unique())

600

In [9]:
print(df_raw['source'].unique()[0])

source-16k/test/sp4899/Lab41-SRI-VOiCES-src-sp4899-ch032639-sg0029.wav


## Split data in 5s

In [10]:
df_raw.columns

Index(['index', 'chapter', 'degrees', 'distractor', 'filename', 'gender',
       'mic', 'query_name', 'room', 'segment', 'source', 'speaker',
       'transcript', 'noisy_length', 'noisy_sr', 'noisy_time', 'source_length',
       'source_sr', 'source_time', 'category'],
      dtype='object')

In [11]:
df_origin = df_raw.loc[:50]
df_origin.head()

,index,chapter,degrees,distractor,filename,gender,mic,query_name,room,segment,source,speaker,transcript,noisy_length,noisy_sr,noisy_time,source_length,source_sr,source_time,category
0,0,32639,160,babb,distant-16k/speech/test/rm4/babb/sp4899/Lab41-...,F,1,Lab41-SRI-VOiCES-rm4-babb-sp4899-ch032639-sg00...,rm4,29,source-16k/test/sp4899/Lab41-SRI-VOiCES-src-sp...,4899,and i got on with the lesson very badly i am s...,255040,16000,15.940,255040,16000,15.940,test
1,1,32639,160,babb,distant-16k/speech/test/rm4/babb/sp4899/Lab41-...,F,5,Lab41-SRI-VOiCES-rm4-babb-sp4899-ch032639-sg00...,rm4,29,source-16k/test/sp4899/Lab41-SRI-VOiCES-src-sp...,4899,and i got on with the lesson very badly i am s...,255040,16000,15.940,255040,16000,15.940,test
2,2,32658,70,babb,distant-16k/speech/test/rm4/babb/sp4899/Lab41-...,F,5,Lab41-SRI-VOiCES-rm4-babb-sp4899-ch032658-sg00...,rm4,12,source-16k/test/sp4899/Lab41-SRI-VOiCES-src-sp...,4899,and ran upstairs to put away the host of littl...,258000,16000,16.125,258000,16000,16.125,test
3,3,32658,70,babb,distant-16k/speech/test/rm4/babb/sp4899/Lab41-...,F,1,Lab41-SRI-VOiCES-rm4-babb-sp4899-ch032658-sg00...,rm4,12,source-16k/test/sp4899/Lab41-SRI-VOiCES-src-sp...,4899,and ran upstairs to put away the host of littl...,258000,16000,16.125,258000,16000,16.125,test
4,4,130550,10,babb,distant-16k/speech/test/rm4/babb/sp1447/Lab41-...,F,1,Lab41-SRI-VOiCES-rm4-babb-sp1447-ch130550-sg00...,rm4,26,source-16k/test/sp1447/Lab41-SRI-VOiCES-src-sp...,1447,he was inexhaustibly voluble,58720,16000,3.670,58720,16000,3.670,test


In [12]:
df_origin.rename(columns={'source': 'source_origin'}, inplace=True)

C:\Users\bobmst\AppData\Local\Temp\ipykernel_34164\490333522.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_origin.rename(columns={'source': 'source_origin'}, inplace=True)


In [13]:
dict(df_origin.iloc[0])

{'index': 0,
 'chapter': 32639,
 'degrees': 160,
 'distractor': 'babb',
 'filename': 'distant-16k/speech/test/rm4/babb/sp4899/Lab41-SRI-VOiCES-rm4-babb-sp4899-ch032639-sg0029-mc01-stu-clo-dg160.wav',
 'gender': 'F',
 'mic': 1,
 'query_name': 'Lab41-SRI-VOiCES-rm4-babb-sp4899-ch032639-sg0029-mc01-stu-clo-dg160',
 'room': 'rm4',
 'segment': 29,
 'source_origin': 'source-16k/test/sp4899/Lab41-SRI-VOiCES-src-sp4899-ch032639-sg0029.wav',
 'speaker': 4899,
 'transcript': 'and i got on with the lesson very badly i am sure maria perry has no gift at teaching little children and i feel quite vexed and disappointed this has not been a profitable sunday and i and now going to bed cheerless and uneasy',
 'noisy_length': 255040,
 'noisy_sr': 16000,
 'noisy_time': 15.94,
 'source_length': 255040,
 'source_sr': 16000,
 'source_time': 15.94,
 'category': 'test'}

In [14]:
df_origin['source_folder_origin'] = df_origin['source_origin'].apply(lambda x: os.path.dirname(x))
df_origin.head()

C:\Users\bobmst\AppData\Local\Temp\ipykernel_34164\2913378930.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_origin['source_folder_origin'] = df_origin['source_origin'].apply(lambda x: os.path.dirname(x))


,index,chapter,degrees,distractor,filename,gender,mic,query_name,room,segment,...,speaker,transcript,noisy_length,noisy_sr,noisy_time,source_length,source_sr,source_time,category,source_folder_origin
0,0,32639,160,babb,distant-16k/speech/test/rm4/babb/sp4899/Lab41-...,F,1,Lab41-SRI-VOiCES-rm4-babb-sp4899-ch032639-sg00...,rm4,29,...,4899,and i got on with the lesson very badly i am s...,255040,16000,15.940,255040,16000,15.940,test,source-16k/test/sp4899
1,1,32639,160,babb,distant-16k/speech/test/rm4/babb/sp4899/Lab41-...,F,5,Lab41-SRI-VOiCES-rm4-babb-sp4899-ch032639-sg00...,rm4,29,...,4899,and i got on with the lesson very badly i am s...,255040,16000,15.940,255040,16000,15.940,test,source-16k/test/sp4899
2,2,32658,70,babb,distant-16k/speech/test/rm4/babb/sp4899/Lab41-...,F,5,Lab41-SRI-VOiCES-rm4-babb-sp4899-ch032658-sg00...,rm4,12,...,4899,and ran upstairs to put away the host of littl...,258000,16000,16.125,258000,16000,16.125,test,source-16k/test/sp4899
3,3,32658,70,babb,distant-16k/speech/test/rm4/babb/sp4899/Lab41-...,F,1,Lab41-SRI-VOiCES-rm4-babb-sp4899-ch032658-sg00...,rm4,12,...,4899,and ran upstairs to put away the host of littl...,258000,16000,16.125,258000,16000,16.125,test,source-16k/test/sp4899
4,4,130550,10,babb,distant-16k/speech/test/rm4/babb/sp1447/Lab41-...,F,1,Lab41-SRI-VOiCES-rm4-babb-sp1447-ch130550-sg00...,rm4,26,...,1447,he was inexhaustibly voluble,58720,16000,3.670,58720,16000,3.670,test,source-16k/test/sp1447


In [15]:
set(df_origin['source_origin'].apply(lambda x: os.path.dirname(x)))

{'source-16k/test/sp0303',
 'source-16k/test/sp0606',
 'source-16k/test/sp0718',
 'source-16k/test/sp1390',
 'source-16k/test/sp1447',
 'source-16k/test/sp1482',
 'source-16k/test/sp1649',
 'source-16k/test/sp2397',
 'source-16k/test/sp2952',
 'source-16k/test/sp3070',
 'source-16k/test/sp4899',
 'source-16k/test/sp6550',
 'source-16k/test/sp8419'}

In [16]:
def replace_first_folder(path, new_root):
    parts = path.split('/')  # Split the path into parts
    if len(parts) > 1:  # Check if there are enough parts to replace
        parts[0] = new_root  # Replace the first part with new_root
        return '/'.join(parts)  # Join the parts back into a path
    else:
        return new_root  # If there's only one part, return the new_root

def read_signal(path):
    signal, sr = librosa.load(path, sr=None)  # Load the file with its original sampling rate
    return signal, sr

def save_segment(signal, sr, start_time, end_time, save_path):
    segment = signal[int(start_time * sr):int(end_time * sr)]
    sf.write(save_path, segment, sr)

def process_file(file_path, new_path, segment_length=5, denoise=True):
    signal, sr = read_signal(file_path)
    # Apply denoise before cutting the segments
    if denoise:
        signal = reduce_noise(signal, sr)
    file_name = os.path.basename(file_path)
    duration = len(signal) / sr
    ls_file_paths = []

    if duration > segment_length:
        num_segments = int(duration // segment_length)
        for i in range(num_segments):
            segment_filename = f"{os.path.splitext(file_name)[0]}_segment_{i}.wav"
            new_file_path = os.path.join(new_path, segment_filename)
            save_segment(signal, sr, i * segment_length, (i + 1) * segment_length, new_file_path)
            ls_file_paths.append(segment_filename)
        # Handle the last segment if there is a remainder
        if duration % segment_length > 0:
            segment_filename = f"{os.path.splitext(file_name)[0]}_segment_{num_segments}.wav"
            new_file_path = os.path.join(new_path, segment_filename)
            save_segment(signal, sr, num_segments * segment_length, duration, new_file_path)
            ls_file_paths.append(segment_filename)
    else:
        # If the file is shorter than or equal to the segment_length, copy it to the new_path
        new_file_path = os.path.join(new_path, file_name)
        sf.write(new_file_path, signal, sr)
        ls_file_paths.append(file_name)
    return ls_file_paths

def process_directory(path, new_path, segment_length=5, denoise=True):
    # Create the new_path directory if it does not exist
    if not os.path.exists(new_path):
        os.makedirs(new_path)

    # List all .wav files in the given directory
    files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f)) and f.endswith('.wav')]

    ls_new_files = []
    # Process each file
    for file in files:
        file_path = os.path.join(path, file)
        ls_new_files += process_file(file_path, new_path, segment_length, denoise)
    return ls_new_files

In [17]:
folder = 'source-16k/test/sp0303'
path = os.path.join(FILE_DIR, folder)
path



'./data/VOiCES_devkit/source-16k/test/sp0303'

In [18]:
os.listdir(path)

['Lab41-SRI-VOiCES-src-sp0303-ch123500-sg0023.wav',
 'Lab41-SRI-VOiCES-src-sp0303-ch123507-sg0017.wav']

In [19]:
replace_first_folder(folder, target_folder)

'5seconds-16k-source/test/sp0303'

In [20]:
new_path = os.path.join(FILE_DIR, replace_first_folder(folder, target_folder))

print(new_path)

./data/VOiCES_devkit/5seconds-16k-source/test/sp0303


In [21]:
replace_first_folder(folder, target_folder)

'5seconds-16k-source/test/sp0303'

In [22]:
process_directory(os.path.join(FILE_DIR,folder), new_path, segment_length=5,denoise=False)

['Lab41-SRI-VOiCES-src-sp0303-ch123500-sg0023_segment_0.wav',
 'Lab41-SRI-VOiCES-src-sp0303-ch123500-sg0023_segment_1.wav',
 'Lab41-SRI-VOiCES-src-sp0303-ch123500-sg0023_segment_2.wav',
 'Lab41-SRI-VOiCES-src-sp0303-ch123500-sg0023_segment_3.wav',
 'Lab41-SRI-VOiCES-src-sp0303-ch123507-sg0017_segment_0.wav',
 'Lab41-SRI-VOiCES-src-sp0303-ch123507-sg0017_segment_1.wav',
 'Lab41-SRI-VOiCES-src-sp0303-ch123507-sg0017_segment_2.wav',
 'Lab41-SRI-VOiCES-src-sp0303-ch123507-sg0017_segment_3.wav']

In [23]:
ls_new_records = []

for folder in tqdm(set(df_raw['source'].apply(lambda x: os.path.dirname(x)))):
    new_record = {}
    new_record['origin_folder'] = folder
    new_record['speaker'] = folder.split('/sp')[-1]
    new_record['category'] = re.search(r'source-16k/([^/]*)', folder).group(1)
    new_folder = replace_first_folder(folder,target_folder)
    
    old_path = os.path.join(FILE_DIR,folder)
    new_path = os.path.join(FILE_DIR, new_folder)
    ls_new_files = process_directory(old_path, new_path, segment_length=5, denoise=False)
    for new_file in ls_new_files:
        new_record['source'] = os.path.join(new_folder,new_file).replace('\\','/')
        ls_new_records.append(new_record)


  5%|▍         | 14/300 [00:00<00:04, 66.35it/s]

100%|██████████| 300/300 [00:06<00:00, 43.49it/s]


In [24]:
df_5s = pd.DataFrame(ls_new_records)
df_5s

,origin_folder,speaker,category,source
0,source-16k/train/sp0242,0242,train,5seconds-16k-source/train/sp0242/Lab41-SRI-VOi...
1,source-16k/train/sp0242,0242,train,5seconds-16k-source/train/sp0242/Lab41-SRI-VOi...
2,source-16k/train/sp0242,0242,train,5seconds-16k-source/train/sp0242/Lab41-SRI-VOi...
3,source-16k/train/sp0242,0242,train,5seconds-16k-source/train/sp0242/Lab41-SRI-VOi...
4,source-16k/train/sp0242,0242,train,5seconds-16k-source/train/sp0242/Lab41-SRI-VOi...
...,...,...,...,...
2322,source-16k/train/sp5456,5456,train,5seconds-16k-source/train/sp5456/Lab41-SRI-VOi...
2323,source-16k/train/sp5456,5456,train,5seconds-16k-source/train/sp5456/Lab41-SRI-VOi...
2324,source-16k/train/sp5456,5456,train,5seconds-16k-source/train/sp5456/Lab41-SRI-VOi...
2325,source-16k/train/sp5456,5456,train,5seconds-16k-source/train/sp5456/Lab41-SRI-VOi...


In [25]:
df_5s.to_pickle(os.path.join(DATA_ADDRESS,'df_5s_source.pkl'))

## Preprocess Test Data

In [26]:
# util.py

def read_signal(path):
    signal, sr = librosa.load(path)
    return signal, sr

def save_pickle(filename,obj):
    with open(filename, 'wb') as f:
        pickle.dump(obj, f)

def load_pickle(filename):
    with open(filename, 'rb') as f:
        obj = pickle.load(f)
    return obj

In [27]:

def process_audios_to_file(*,ls_files,save_dir,n_interval=1000,n_length=15, n_fft=2048, n_mels=128, n_mfcc=20, denoise=False):
    n_count = 0
    n_start = 0
    ls_spectrogram = []
    ls_mel = []
    ls_mfcc = []

    
    for i, file_name in enumerate(tqdm(ls_files)):
        n_count += 1
        # print(i,file_name)
        s,sr = read_signal(os.path.join(FILE_DIR,file_name))
        spectrogram, melspectrogram, mfcc = process_audio(s,sr, length=n_length, n_fft=n_fft, n_mels=n_mels, n_mfcc=n_mfcc,denoise=denoise)
        ls_spectrogram.append(spectrogram)
        ls_mel.append(melspectrogram)
        ls_mfcc.append(mfcc)

        if n_count>= n_interval:
            # print(str(n)+'-'+str(i))

            # save to files 
            save_pickle(save_dir+f'source_spectrogram_len{n_length}_fft{n_fft}_mels{n_mels}_mfcc{n_mfcc}_{str(n_start)}-{str(i)}.pkl',np.array(ls_spectrogram))
            save_pickle(save_dir+f'source_mel_len{n_length}_fft{n_fft}_mels{n_mels}_mfcc{n_mfcc}_{str(n_start)}-{str(i)}.pkl',np.array(ls_mel))
            save_pickle(save_dir+f'source_mfcc_len{n_length}_fft{n_fft}_mels{n_mels}_mfcc{n_mfcc}_{str(n_start)}-{str(i)}.pkl',np.array(ls_mfcc))

            n_start=i+1
            # reset list
            n_count = 0
            ls_spectrogram.clear()
            ls_mel.clear()
            ls_mfcc.clear()
    else:
        save_pickle(save_dir+f'source_spectrogram_len{n_length}_fft{n_fft}_mels{n_mels}_mfcc{n_mfcc}_{str(n_start)}-{str(i)}.pkl',np.array(ls_spectrogram))
        save_pickle(save_dir+f'source_mel_len{n_length}_fft{n_fft}_mels{n_mels}_mfcc{n_mfcc}_{str(n_start)}-{str(i)}.pkl',np.array(ls_mel))
        save_pickle(save_dir+f'source_mfcc_len{n_length}_fft{n_fft}_mels{n_mels}_mfcc{n_mfcc}_{str(n_start)}-{str(i)}.pkl',np.array(ls_mfcc))

# Be ware of the index numbers if use concurrent for loop

In [28]:
# set parameters
n_interval = 500

n_length = 5
n_fft = 2048
n_mels = 128
n_mfcc = 17



In [29]:
# Process test files
save_dir = data_address+'preprocessed/test/'
process_audios_to_file(ls_files = df_5s[df_5s['category']=='test']['source'],
save_dir=save_dir,
n_interval=n_interval,n_length=n_length, n_fft=n_fft, n_mels=n_mels, n_mfcc=n_mfcc)

100%|██████████| 775/775 [00:09<00:00, 85.74it/s] 


In [30]:
# Process train files
save_dir = data_address+'preprocessed/train/'
process_audios_to_file(ls_files = df_5s[df_5s['category']=='train']['source'],
save_dir=save_dir,
n_interval=n_interval,n_length=n_length, n_fft=n_fft, n_mels=n_mels, n_mfcc=n_mfcc)

100%|██████████| 1552/1552 [00:18<00:00, 82.43it/s] 


## Check load data

In [31]:
save_dir = data_address+'preprocessed/test/'
data_mfcc = load_pickle(save_dir + 'source_mfcc_len5_fft2048_mels128_mfcc17_0-499.pkl')

In [32]:
data_mfcc.shape

(500, 17, 216)

In [33]:
data_mfcc[0].shape

(17, 216)

In [34]:
data_mel = load_pickle(save_dir + 'mel_len5_fft2048_mels128_mfcc17_0-499.pkl')

In [35]:
data_mel.shape

(500, 128, 216)

In [36]:
data_spec = load_pickle(save_dir + 'spectrogram_len5_fft2048_mels128_mfcc17_0-499.pkl')

In [37]:
data_spec.shape

(500, 1025, 216)